In [1]:
library(IRdisplay)
display_html("<style>.container { width:100% !important; }</style>")

In [2]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(purrr))
suppressPackageStartupMessages(library(deMULTIplex))
suppressPackageStartupMessages(library(optparse))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))

In [3]:
opts <- list()
opts$inputdir <- "/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/"
opts$settings <- "/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/gastruloids_scRNAseq/pipeline/processing/settings.R"
opts$experiment <- "MULTI/exp3C_d4_d4.5"
opts$sample <- "sample1"
source(opts$settings)

In [4]:
full_outbase <- paste0(opts$inputdir, opts$experiment, "/")
final.calls.rescued <- readRDS(paste0(full_outbase, opts$sample, "_finalCalls.rds"))
final.calls.rescued

Warning message in gzfile(file, "rb"):
“cannot open compressed file '/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/MULTI/exp3C_d4_d4.5/sample1_finalCalls.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [ ]:
bar.table <- readRDS(paste0(opts$inputdir,opts$experiment,"/",opts$sample,"_barTable.rds"))

In [ ]:
bar.table$MULTI_class <- final.calls.rescued[rownames(bar.table)]

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

p <- ggplot(bar.table, aes(x=nUMI)) +
    geom_density(fill="darkgrey") +
    scale_x_continuous(trans="log2") +
    theme_bw()
print(p)

p <- ggplot(bar.table, aes(x=nUMI, fill=MULTI_class)) +
    geom_density(alpha=0.4) +
    scale_x_continuous(trans="log2") +
    theme_bw()
print(p)

p <- ggplot(bar.table, aes(x=nUMI_total)) +
    geom_density(fill="darkgrey") +
    scale_x_continuous(trans="log2") +
    theme_bw()
print(p)

p <- ggplot(bar.table, aes(x=nUMI_total, fill=MULTI_class)) +
    geom_density(alpha=0.4) +
    scale_x_continuous(trans="log2") +
    theme_bw()
print(p)

options(repr.plot.width=10, repr.plot.height=4)
p <- ggplot(bar.table, aes(y=nUMI_total, x=MULTI_class, fill=MULTI_class)) +
    geom_boxplot() +
    scale_y_continuous(trans="log2") +
    theme_classic() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

print(p)

In [ ]:
## Visualize barcode space
bar.tsne <- barTSNE(bar.table[,io$bars_used[[opts$experiment]][[opts$sample]]])
bar.tsne$MULTI_class <- final.calls.rescued[rownames(bar.tsne)]
options(repr.plot.width=5, repr.plot.height=5)
for (i in 3:(ncol(bar.tsne)-1)) {
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = bar.tsne[,i])) +
        geom_point() +
        scale_color_gradient(low = "black", high = "red") +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = MULTI_class == colnames(bar.tsne)[i])) +
        geom_point() +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
}

In [ ]:
## Visualize barcode space
cols <- io$bars_used[[opts$experiment]][[opts$sample]]
cols <- cols[!(cols %in% c(6, 12, 24))]
bar.tsne.new <- barTSNE(bar.table[,cols])
bar.tsne.new$MULTI_class <- final.calls.rescued[rownames(bar.tsne.new)]
options(repr.plot.width=5, repr.plot.height=5)
for (i in 3:(ncol(bar.tsne.new)-1)) {
    p <- ggplot(bar.tsne.new, aes(x = TSNE1, y = TSNE2, color = bar.tsne.new[,i])) +
        geom_point() +
        scale_color_gradient(low = "black", high = "red") +
        ggtitle(colnames(bar.tsne.new)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    p <- ggplot(bar.tsne.new, aes(x = TSNE1, y = TSNE2, color = MULTI_class == colnames(bar.tsne.new)[i])) +
        geom_point() +
        ggtitle(colnames(bar.tsne.new)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
}

In [ ]:
bar.table.full <- bar.table[,1:96]
good.bars <- paste("Bar",io$bars_used[[opts$experiment]][[opts$sample]],sep="")  # Barcodes 1:24 were detected
#good.bars <- good.bars[!(good.bars %in% c('Bar24', 'Bar6', 'Bar12'))]
bar.table <- bar.table.full[, good.bars]  # Remove missing bars and summary columns

bar.table_sweep.list <- list()
n <- 0
for (q in seq(0.01, 0.99, by=0.02)) {
  #print(q)
  n <- n + 1
  bar.table_sweep.list[[n]] <- classifyCells(bar.table, q=q)
  names(bar.table_sweep.list)[n] <- paste("q=",q,sep="")
}

In [ ]:
## Round 1 -----------------------------------------------------------------------------------------------------
## Perform Quantile Sweep

i <- 1
bar.table_sweep.list <- list()
n <- 0
for (q in seq(0.01, 0.99, by=0.02)) {
  n <- n + 1
  bar.table_sweep.list[[n]] <- classifyCells(bar.table, q=q)
  names(bar.table_sweep.list)[n] <- paste("q=",q,sep="")
}

## Identify ideal inter-maxima quantile to set barcode-specific thresholds

threshold.results <- findThresh(call.list=bar.table_sweep.list)
options(repr.plot.width=6, repr.plot.height=4)
p <- ggplot(data=threshold.results$res, aes(x=q, y=Proportion, color=Subset)) + geom_line() + theme(legend.position = "none") +
  geom_vline(xintercept=threshold.results$extrema, lty=2) + scale_color_manual(values=c("red","black","blue"))
print(p)

In [ ]:
## Finalize round 1 classifications, remove negative cells
round.calls <- classifyCells(bar.table, q=findQ(threshold.results$res, threshold.results$extrema))
neg.cells <- names(round.calls)[which(round.calls == "Negative")]
bar.table <- bar.table[-which(rownames(bar.table) %in% neg.cells), ]

options(repr.plot.width=6, repr.plot.height=4)
while (length(names(round.calls)[which(round.calls == "Negative")]) > 0) {
    
    i <- i+1
    bar.table_sweep.list <- list()
    n <- 0
    for (q in seq(0.01, 0.99, by=0.02)) {
      #print(q)
      n <- n + 1
      bar.table_sweep.list[[n]] <- classifyCells(bar.table, q=q)
      names(bar.table_sweep.list)[n] <- paste("q=",q,sep="")
    }
    
    error1 <- FALSE
    tryCatch(threshold.results <- findThresh(call.list=bar.table_sweep.list),
             error = function(e) { error1 <<- TRUE}
            )
    if (error1==TRUE) {
        break
    }

    threshold.results <- findThresh(call.list=bar.table_sweep.list)
    p <- ggplot(data=threshold.results$res, aes(x=q, y=Proportion, color=Subset)) + geom_line() + theme(legend.position = "none") +
      geom_vline(xintercept=threshold.results$extrema, lty=2) + scale_color_manual(values=c("red","black","blue"))
    print(p)
    round.calls <- classifyCells(bar.table, q=findQ(threshold.results$res, threshold.results$extrema))
    neg.cells <- c(neg.cells, names(round.calls)[which(round.calls == "Negative")])
    bar.table <- bar.table[-which(rownames(bar.table) %in% neg.cells), ]
    
}

In [ ]:
final.calls <- c(round.calls, rep("Negative",length(neg.cells)))
names(final.calls) <- c(names(round.calls),neg.cells)

In [ ]:
bar.tsne$MULTI_class_new <- final.calls[rownames(bar.tsne)]
for (i in 3:(ncol(bar.tsne)-2)) {
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = bar.tsne[,i])) +
        geom_point() +
        scale_color_gradient(low = "black", high = "red") +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = MULTI_class_new == colnames(bar.tsne)[i])) +
        geom_point() +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
}

In [ ]:
bar.table.hm <- bar.table.full[names(final.calls),good.bars]
bar.table.hm <- data.table(bar.table.hm)
rownames(bar.table.hm) <- names(final.calls)

argmax <- max.col(bar.table.hm)
bar.table.hm.sorted <- bar.table.hm[sort(argmax, index.return=TRUE)$ix,]
rownames(bar.table.hm.sorted) <- rownames(bar.table.hm)[sort(argmax, index.return=TRUE)$ix]
argmax2 <- max.col(replace(as.matrix(bar.table.hm.sorted), cbind(1:dim(bar.table.hm.sorted)[1], max.col(as.matrix(bar.table.hm.sorted))), -Inf))

sorted_argmax <- max.col(bar.table.hm.sorted)
rns <- rownames(bar.table.hm.sorted)
for (i in unique(argmax)) {
    bar.table.hm.sorted[sorted_argmax==i,] = bar.table.hm.sorted[sorted_argmax==i,][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix,]
    
    rns[sorted_argmax==i] <- rns[sorted_argmax==i][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix]
}
bar.table.hm.sorted <- as.matrix(bar.table.hm.sorted)
rownames(bar.table.hm.sorted) <- rns

to.plot <- melt(bar.table.hm.sorted, varnames = c("barcode", "MULTI_class"))
to.plot$final_call <- unname(final.calls[as.character(to.plot$barcode)])

options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

options(repr.plot.width=40, repr.plot.height=40)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  facet_wrap(~final_call, scales = "free_y")
print(p)

In [ ]:
table(final.calls)

In [ ]:
good.bars[!(good.bars %in% unique(final.calls))]

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
## Perform semi-supervised negative cell reclassification
if (!error1) {
    
    error2 <- FALSE
    tryCatch(reclass.cells <- findReclassCells(bar.table.full[,good.bars], names(final.calls)[which(final.calls=="Negative")]),
             error = function(e) { error2 <<- TRUE}
            )
    
    if (!error2) {
    
        reclass.cells <- findReclassCells(bar.table.full[,good.bars], names(final.calls)[which(final.calls=="Negative")])
        error3 <- FALSE
        tryCatch(reclass.res <- rescueCells(bar.table.full[,good.bars], final.calls, reclass.cells),
                 error = function(e) { error3 <<- TRUE}
                )
        if (!error3) {

            reclass.res <- rescueCells(bar.table.full[,good.bars], final.calls, reclass.cells)

            ## Visualize Results
            p <- ggplot(reclass.res[-1, ], aes(x=ClassStability, y=MatchRate_mean)) + 
                geom_point() + xlim(c(nrow(reclass.res)-1,1)) + 
                ylim(c(0,1.05)) +
                geom_errorbar(aes(ymin=MatchRate_mean-MatchRate_sd, ymax=MatchRate_mean+MatchRate_sd), width=.1) +
                geom_hline(yintercept = reclass.res$MatchRate_mean[1], color="red") +
                geom_hline(yintercept = reclass.res$MatchRate_mean[1]+3*reclass.res$MatchRate_sd[1], color="red",lty=2) +
                geom_hline(yintercept = reclass.res$MatchRate_mean[1]-3*reclass.res$MatchRate_sd[1], color="red",lty=2) +
                theme_classic()
            print(p)

            ## Finalize negative cell rescue results
            final.calls.rescued <- final.calls
            #rescue.ind <- which(reclass.cells$ClassStability >= io$reclass_stability[[opts$experiment]][[opts$sample]]) ## Note: Value will be dataset-specific
            rescue.ind <- which(reclass.cells$ClassStability >= 7) ## Note: Value will be dataset-specific
            final.calls.rescued[rownames(reclass.cells)[rescue.ind]] <- reclass.cells$Reclassification[rescue.ind]

        }
    }
}

In [ ]:
table(final.calls)

In [ ]:
table(final.calls.rescued)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
bar.tsne$MULTI_class_rescued <- final.calls.rescued[rownames(bar.tsne)]
for (i in 3:(ncol(bar.tsne)-3)) {
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = bar.tsne[,i])) +
        geom_point() +
        scale_color_gradient(low = "black", high = "red") +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = MULTI_class_rescued == colnames(bar.tsne)[i])) +
        geom_point() +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
}

Bar15, Bar20, Bar52, and Bar61 still low.

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
to.plot <- bar.tsne %>%
    group_by(MULTI_class, MULTI_class_rescued) %>%
    summarise(n=n())
p <- ggplot(to.plot, aes(MULTI_class, MULTI_class_rescued, fill= n)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  scale_fill_gradient(low="white", high="#2171b5", trans = 'log2') +
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_bw() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

to.plot <- bar.tsne %>%
    group_by(MULTI_class, MULTI_class_new) %>%
    summarise(n=n())
p <- ggplot(to.plot, aes(MULTI_class, MULTI_class_new, fill= n)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  scale_fill_gradient(low="white", high="#2171b5", trans = 'log2') +
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_bw() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

In [ ]:
bar.table.hm <- bar.table.full[names(final.calls.rescued),good.bars]
bar.table.hm <- data.table(bar.table.hm)
rownames(bar.table.hm) <- names(final.calls.rescued)

argmax <- max.col(bar.table.hm)
bar.table.hm.sorted <- bar.table.hm[sort(argmax, index.return=TRUE)$ix,]
rownames(bar.table.hm.sorted) <- rownames(bar.table.hm)[sort(argmax, index.return=TRUE)$ix]
argmax2 <- max.col(replace(as.matrix(bar.table.hm.sorted), cbind(1:dim(bar.table.hm.sorted)[1], max.col(as.matrix(bar.table.hm.sorted))), -Inf))

sorted_argmax <- max.col(bar.table.hm.sorted)
rns <- rownames(bar.table.hm.sorted)
for (i in unique(argmax)) {
    bar.table.hm.sorted[sorted_argmax==i,] = bar.table.hm.sorted[sorted_argmax==i,][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix,]
    
    rns[sorted_argmax==i] <- rns[sorted_argmax==i][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix]
}
bar.table.hm.sorted <- as.matrix(bar.table.hm.sorted)
rownames(bar.table.hm.sorted) <- rns

to.plot <- melt(bar.table.hm.sorted, varnames = c("barcode", "MULTI_class"))
to.plot$final_call <- unname(final.calls.rescued[as.character(to.plot$barcode)])

options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

options(repr.plot.width=40, repr.plot.height=40)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  facet_wrap(~final_call, scales = "free_y")
print(p)

In [ ]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggalluvial))
suppressPackageStartupMessages(library(ggrepel))
library(gtools)
library(ggsignif)
source("/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/gastruloids_scRNAseq/plotting_settings.R")

In [ ]:
g_meta <- fread('/nfs/research/marioni/Leah/gastrulation_epigenetics/gastruloid_characterisation/data_dir/processed_files_Leah/scRNAseq/plot_df_joined.txt.gz')
g_meta[MULTI_class %in% as.vector(unlist(neural)), gastr_type := "neural"]
g_meta[MULTI_class %in% as.vector(unlist(meso)), gastr_type := "meso"]
g_meta[MULTI_class %in% as.vector(unlist(intermediate)), gastr_type := "intermediate"]

g_meta <- g_meta[batch == 'exp3B_d4_d4.5_sample2']

In [ ]:
MULTI_class1 <- final.calls
names(MULTI_class1) <- paste0(names(MULTI_class1), '-1')

#MULTI_class2 <- final.calls2
#names(MULTI_class2) <- paste0(names(MULTI_class2), '-1')

MULTI_class3 <- final.calls.rescued
names(MULTI_class3) <- paste0(names(MULTI_class3), '-1')

g_meta$MULTI_class1 <- unname(MULTI_class1[g_meta$barcode])
#g_meta$MULTI_class2 <- unname(MULTI_class2[g_meta$barcode])
g_meta$MULTI_class3 <- unname(MULTI_class3[g_meta$barcode])

g_meta[paste0(MULTI_class3, '_exp3B_d4_d4.5') %in% as.vector(unlist(neural)), gastr_type3 := "neural"]
g_meta[paste0(MULTI_class3, '_exp3B_d4_d4.5') %in% as.vector(unlist(meso)), gastr_type3 := "meso"]
g_meta[paste0(MULTI_class3, '_exp3B_d4_d4.5') %in% as.vector(unlist(intermediate)), gastr_type3 := "intermediate"]
g_meta[MULTI_class3 %in% c("Negative", "Doublet"), gastr_type3 := MULTI_class3]
#g_meta[MULTI_class3 == 'Bar18', gastr_type3 := "meso"]

In [ ]:
gastr_order_long <- c( "Bar49_exp3B_d4_d4.5", "Bar2_exp3B_d4_d4.5", "Bar25_exp3B_d4_d4.5", "Bar22_exp3B_d4_d4.5", "Bar6_exp3B_d4_d4.5", "Bar30_exp3B_d4_d4.5", "Bar5_exp3B_d4_d4.5", "Bar52_exp3B_d4_d4.5",
                 "Bar56_exp3B_d4_d4.5", "Bar55_exp3B_d4_d4.5", "Bar12_exp3B_d4_d4.5", "Bar28_exp3B_d4_d4.5", "Bar47_exp3B_d4_d4.5", "Bar19_exp3B_d4_d4.5", "Bar15_exp3B_d4_d4.5", "Bar13_exp3B_d4_d4.5",
                 "Bar61_exp3B_d4_d4.5", "Bar45_exp3B_d4_d4.5", "Bar14_exp3B_d4_d4.5", "Bar39_exp3B_d4_d4.5", "Bar66_exp3B_d4_d4.5", "Bar59_exp3B_d4_d4.5", "Bar65_exp3B_d4_d4.5", "Bar41_exp3B_d4_d4.5"
)

options(repr.plot.width=25, repr.plot.height=12)
to.plot <- g_meta[,.N,by=c("cluster","MULTI_class")]
to.plot$MULTI_class = factor(to.plot$MULTI_class, levels=gastr_order_long)
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- barplot.pub(to.plot[!is.na(cluster)], x="cluster", colors=cluster_colours) +
  facet_wrap(~MULTI_class, nrow=2, scales="free_x")
print(p)

In [ ]:
gastr_order_short <- c( "Bar49", "Bar2", "Bar25", "Bar22", "Bar6", "Bar30", "Bar5", "Bar52",
                 "Bar56", "Bar55", "Bar12", "Bar28", "Bar47", "Bar19", "Bar15", "Bar13",
                 "Bar61", "Bar45", "Bar14", "Bar39", "Bar66", "Bar59", "Bar65", "Bar41"
)

options(repr.plot.width=25, repr.plot.height=12)
to.plot <- g_meta[,.N,by=c("cluster","MULTI_class1")]
to.plot$MULTI_class1 = factor(to.plot$MULTI_class1, levels=gastr_order_short)
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- barplot.pub(to.plot[!is.na(cluster)], x="cluster", colors=cluster_colours) +
  facet_wrap(~MULTI_class1, nrow=2, scales="free_x")
print(p)

#to.plot <- g_meta[,.N,by=c("cluster","MULTI_class2")]
#to.plot$MULTI_class2 = factor(to.plot$MULTI_class2, levels=gastr_order_short)
#to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
#p <- barplot.pub(to.plot[!is.na(cluster)], x="cluster", colors=cluster_colours) +
#  facet_wrap(~MULTI_class2, nrow=2, scales="free_x") +
#  ggtitle('d3')
#print(p)

to.plot <- g_meta[,.N,by=c("cluster","MULTI_class3")]
to.plot$MULTI_class3 = factor(to.plot$MULTI_class3, levels=gastr_order_short)
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- barplot.pub(to.plot[!is.na(cluster)], x="cluster", colors=cluster_colours) +
  facet_wrap(~MULTI_class3, nrow=2, scales="free_x")
print(p)

In [ ]:
g_meta[!(MULTI_class %in% c('Doublet', 'Negative')),MULTI_agree := MULTI_class == paste0(MULTI_class3, '_exp3B_d4_d4.5')]
g_meta[(MULTI_class %in% c('Doublet', 'Negative')),MULTI_agree := MULTI_class == MULTI_class3]

In [ ]:
to.plot <- g_meta[,.N,by=c("cluster", "MULTI_class3", "MULTI_agree")]
to.plot$MULTI_class3 = factor(to.plot$MULTI_class3, levels=gastr_order_short)
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- ggplot(to.plot, aes(x = cluster, y = N)) + scale_x_discrete(drop = FALSE) + 
    labs(y = "Number of cells") +
    theme_classic() +
    theme(strip.background = element_blank(),
          strip.text = element_text(color = "black", size = rel(1.3)),
          axis.title.x = element_text(color = "black", size = rel(1.1)),
          axis.title.y = element_blank(), 
          axis.text.y = element_text(size = rel(1.3), color = "black"), 
          axis.text.x = element_text(size = rel(1.1), color = "black")) +
    geom_bar(stat = "identity", aes(fill = MULTI_agree)) +
    coord_flip() +
    facet_wrap(~MULTI_class3, nrow=2, scales="free_x")
print(p)
p <- ggplot(to.plot, aes(x = cluster, y = N)) + scale_x_discrete(drop = FALSE) + 
    labs(y = "Number of cells") +
    theme_classic() +
    theme(strip.background = element_blank(),
          strip.text = element_text(color = "black", size = rel(1.3)),
          axis.title.x = element_text(color = "black", size = rel(1.1)),
          axis.title.y = element_blank(), 
          axis.text.y = element_text(size = rel(1.3), color = "black"), 
          axis.text.x = element_text(size = rel(1.1), color = "black")) +
    geom_bar(stat = "identity", position="fill", aes(fill = MULTI_agree)) +
    coord_flip() +
    facet_wrap(~MULTI_class3, nrow=2, scales="free_x")
print(p)

to.plot <- g_meta[, sum := .N, by = c("MULTI_class3", "cluster", "gastr_type3")][, prop := .N, by = c("MULTI_class3", "cluster", "MULTI_agree", "gastr_type3")][, prop := prop/sum][, sum := NULL][,cell := NULL] %>% unique
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- ggplot(to.plot, aes(x=cluster, y=prop, fill=gastr_type3)) +
  geom_boxplot() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  #scale_y_continuous(trans='log2') +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

In [ ]:
g_meta[,.N,by=c("MULTI_agree", "MULTI_class3", "gastr_type3")]

In [ ]:
to.plot <- g_meta[,.N,by=c("cluster", "MULTI_class", "MULTI_agree")]
to.plot$MULTI_class = factor(to.plot$MULTI_class, levels=gastr_order_long)
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- ggplot(to.plot, aes(x = cluster, y = N)) + scale_x_discrete(drop = FALSE) + 
    labs(y = "Number of cells") +
    theme_classic() +
    theme(strip.background = element_blank(),
          strip.text = element_text(color = "black", size = rel(1.3)),
          axis.title.x = element_text(color = "black", size = rel(1.1)),
          axis.title.y = element_blank(), 
          axis.text.y = element_text(size = rel(1.3), color = "black"), 
          axis.text.x = element_text(size = rel(1.1), color = "black")) +
    geom_bar(stat = "identity", aes(fill = MULTI_agree)) +
    coord_flip() +
    facet_wrap(~MULTI_class, nrow=2, scales="free_x")
print(p)

p <- ggplot(to.plot, aes(x = cluster, y = N)) + scale_x_discrete(drop = FALSE) + 
    labs(y = "Number of cells") +
    theme_classic() +
    theme(strip.background = element_blank(),
          strip.text = element_text(color = "black", size = rel(1.3)),
          axis.title.x = element_text(color = "black", size = rel(1.1)),
          axis.title.y = element_blank(), 
          axis.text.y = element_text(size = rel(1.3), color = "black"), 
          axis.text.x = element_text(size = rel(1.1), color = "black")) +
    geom_bar(stat = "identity", position="fill", aes(fill = MULTI_agree)) +
    coord_flip() +
    facet_wrap(~MULTI_class, nrow=2, scales="free_x")
print(p)

to.plot <- g_meta[,.N,by=c("cluster", "gastr_type", "MULTI_agree")]
to.plot$cluster = factor(to.plot$cluster, levels=rev(cluster_order))
p <- ggplot(to.plot, aes(x = cluster, y = N)) + scale_x_discrete(drop = FALSE) + 
    labs(y = "Number of cells") +
    theme_classic() +
    theme(strip.background = element_blank(),
          strip.text = element_text(color = "black", size = rel(1.3)),
          axis.title.x = element_text(color = "black", size = rel(1.1)),
          axis.title.y = element_blank(), 
          axis.text.y = element_text(size = rel(1.3), color = "black"), 
          axis.text.x = element_text(size = rel(1.1), color = "black")) +
    geom_bar(stat = "identity", position="fill", aes(fill = MULTI_agree)) +
    coord_flip() +
    facet_wrap(~gastr_type, nrow=2, scales="free_x")
print(p)

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)
bar.tsne$cluster <- g_meta[match(paste0(rownames(bar.tsne), '-1'), g_meta$barcode)]$cluster
for (i in 3:(ncol(bar.tsne)-4)) {
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = bar.tsne[,i])) +
        geom_point() +
        scale_color_gradient(low = "black", high = "red") +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    p <- ggplot(bar.tsne, aes(x = TSNE1, y = TSNE2, color = MULTI_class_rescued == colnames(bar.tsne)[i])) +
        geom_point() +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    to.plot <- as.data.table(bar.tsne)
    p <- ggplot(data=to.plot[MULTI_class_rescued == colnames(bar.tsne)[i]], mapping = aes(x = TSNE1, y = TSNE2, color = cluster)) +
      geom_point(data=select(to.plot,-cluster), colour="grey", size=1, alpha=1) +
      geom_point(size=1, alpha=1) +
        scale_colour_manual(values = cluster_colours, name = "Cluster") +
        ggtitle(colnames(bar.tsne)[i]) +
        theme_bw() +
        theme(legend.position = "none") 
    print(p)
    
}

In [ ]:
bar.table.hm <- bar.table.full[names(final.calls.rescued),good.bars]
bar.table.hm <- data.table(bar.table.hm)
rownames(bar.table.hm) <- names(final.calls.rescued)

argmax <- max.col(bar.table.hm)
bar.table.hm.sorted <- bar.table.hm[sort(argmax, index.return=TRUE)$ix,]
rownames(bar.table.hm.sorted) <- rownames(bar.table.hm)[sort(argmax, index.return=TRUE)$ix]
argmax2 <- max.col(replace(as.matrix(bar.table.hm.sorted), cbind(1:dim(bar.table.hm.sorted)[1], max.col(as.matrix(bar.table.hm.sorted))), -Inf))

sorted_argmax <- max.col(bar.table.hm.sorted)
rns <- rownames(bar.table.hm.sorted)
for (i in unique(argmax)) {
    bar.table.hm.sorted[sorted_argmax==i,] = bar.table.hm.sorted[sorted_argmax==i,][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix,]
    
    rns[sorted_argmax==i] <- rns[sorted_argmax==i][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix]
}
bar.table.hm.sorted <- as.matrix(bar.table.hm.sorted)
rownames(bar.table.hm.sorted) <- rns

to.plot <- melt(bar.table.hm.sorted, varnames = c("barcode", "MULTI_class"))
to.plot$final_call <- unname(final.calls.rescued[as.character(to.plot$barcode)])
#to.plot$cluster <- factor(g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster, levels=cluster_order)
to.plot$cluster <- g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster
to.plot <- to.plot[order(match(to.plot$cluster, cluster_order)),]
to.plot$barcode <- factor(to.plot$barcode, levels=unique(as.character(to.plot$barcode)))
to.plot$cluster <- factor(to.plot$cluster, levels=cluster_order)

options(repr.plot.width=10, repr.plot.height=10)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
print(p)

options(repr.plot.width=40, repr.plot.height=40)
p <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  facet_wrap(~final_call, scales = "free_y")
print(p)

In [ ]:
library(gridExtra)
library(grid)

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)

p1 <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  scale_fill_continuous(type = "viridis", trans = 'log2') +
  theme_classic() +
  theme(legend.position = "None",
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
p2 <- ggplot(to.plot, aes(1, barcode, fill= cluster)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
grid.arrange(p1,p2,ncol=2)

for (fc in unique(to.plot$final_call)) {
    
    p1 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    p2 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(1, barcode, fill= cluster)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      #scale_fill_continuous(type = "viridis", trans = 'log2') +
      scale_fill_manual(values = cluster_colours, name = "Cluster") +
      theme_classic() +
      theme(axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    print(grid.arrange(p1,p2,ncol=2,
                       top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    
    p1 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    
    
    if (nrow(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))]) > 0) {
        p2 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(1, barcode, fill= cluster)) + 
          geom_tile() +
          #scale_fill_gradient( trans = 'log2' ) +
          #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
          #scale_fill_continuous(type = "viridis", trans = 'log2') +
          scale_fill_manual(values = cluster_colours, name = "Cluster") +
          theme_classic() +
          theme(axis.text.y=element_blank(),
                axis.ticks.y=element_blank(),
                axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
        print(grid.arrange(p1,p2,ncol=2,
                           top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    }
    
}

options(repr.plot.width=20, repr.plot.height=15)
to.plot$MULTI_class = factor(to.plot$MULTI_class, levels=gastr_order_short)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_violin() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_boxplot() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)

In [ ]:
library(matrixStats)

In [ ]:
bar.table.hm <- bar.table.full[names(final.calls.rescued),good.bars]
bar.table.hm <- log2(bar.table.hm + 1)
bar.table.hm <- (bar.table.hm - rowMeans(bar.table.hm))/sqrt(rowVars(as.matrix(bar.table.hm)))
bar.table.hm <- data.table(bar.table.hm)
rownames(bar.table.hm) <- names(final.calls.rescued)

argmax <- max.col(bar.table.hm)
bar.table.hm.sorted <- bar.table.hm[sort(argmax, index.return=TRUE)$ix,]
rownames(bar.table.hm.sorted) <- rownames(bar.table.hm)[sort(argmax, index.return=TRUE)$ix]
argmax2 <- max.col(replace(as.matrix(bar.table.hm.sorted), cbind(1:dim(bar.table.hm.sorted)[1], max.col(as.matrix(bar.table.hm.sorted))), -Inf))

sorted_argmax <- max.col(bar.table.hm.sorted)
rns <- rownames(bar.table.hm.sorted)
for (i in unique(argmax)) {
    bar.table.hm.sorted[sorted_argmax==i,] = bar.table.hm.sorted[sorted_argmax==i,][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix,]
    
    rns[sorted_argmax==i] <- rns[sorted_argmax==i][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix]
}
bar.table.hm.sorted <- as.matrix(bar.table.hm.sorted)
rownames(bar.table.hm.sorted) <- rns

to.plot <- melt(bar.table.hm.sorted, varnames = c("barcode", "MULTI_class"))
to.plot$final_call <- unname(final.calls.rescued[as.character(to.plot$barcode)])
#to.plot$cluster <- factor(g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster, levels=cluster_order)
to.plot$cluster <- g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster
to.plot <- to.plot[order(match(to.plot$cluster, cluster_order)),]
to.plot$barcode <- factor(to.plot$barcode, levels=unique(as.character(to.plot$barcode)))
to.plot$cluster <- factor(to.plot$cluster, levels=cluster_order)

options(repr.plot.width=20, repr.plot.height=10)

p1 <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  scale_fill_continuous(type = "viridis") +
  theme_classic() +
  theme(legend.position = "None",
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
p2 <- ggplot(to.plot, aes(1, barcode, fill= cluster)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
grid.arrange(p1,p2,ncol=2)

for (fc in unique(to.plot$final_call)) {
    
    p1 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    p2 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(1, barcode, fill= cluster)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      #scale_fill_continuous(type = "viridis", trans = 'log2') +
      scale_fill_manual(values = cluster_colours, name = "Cluster") +
      theme_classic() +
      theme(axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    print(grid.arrange(p1,p2,ncol=2,
                       top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    
    p1 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    
    
    if (nrow(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))]) > 0) {
        p2 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(1, barcode, fill= cluster)) + 
          geom_tile() +
          #scale_fill_gradient( trans = 'log2' ) +
          #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
          #scale_fill_continuous(type = "viridis", trans = 'log2') +
          scale_fill_manual(values = cluster_colours, name = "Cluster") +
          theme_classic() +
          theme(axis.text.y=element_blank(),
                axis.ticks.y=element_blank(),
                axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
        print(grid.arrange(p1,p2,ncol=2,
                           top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    }
    
}

options(repr.plot.width=20, repr.plot.height=15)
to.plot$MULTI_class = factor(to.plot$MULTI_class, levels=gastr_order_short)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_violin() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_boxplot() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)

In [ ]:
bar.table.hm <- bar.table.full[names(final.calls.rescued),good.bars]
bar.table.hm <- log2(bar.table.hm + 1)
bar.table.hm <- t(t(bar.table.hm)/colSums(bar.table.hm))
bar.table.hm <- (bar.table.hm - rowMeans(bar.table.hm))/sqrt(rowVars(as.matrix(bar.table.hm)))
bar.table.hm <- data.table(bar.table.hm)
rownames(bar.table.hm) <- names(final.calls.rescued)

argmax <- max.col(bar.table.hm)
bar.table.hm.sorted <- bar.table.hm[sort(argmax, index.return=TRUE)$ix,]
rownames(bar.table.hm.sorted) <- rownames(bar.table.hm)[sort(argmax, index.return=TRUE)$ix]
argmax2 <- max.col(replace(as.matrix(bar.table.hm.sorted), cbind(1:dim(bar.table.hm.sorted)[1], max.col(as.matrix(bar.table.hm.sorted))), -Inf))

sorted_argmax <- max.col(bar.table.hm.sorted)
rns <- rownames(bar.table.hm.sorted)
for (i in unique(argmax)) {
    bar.table.hm.sorted[sorted_argmax==i,] = bar.table.hm.sorted[sorted_argmax==i,][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix,]
    
    rns[sorted_argmax==i] <- rns[sorted_argmax==i][sort(argmax2[sorted_argmax==i], index.return=TRUE)$ix]
}
bar.table.hm.sorted <- as.matrix(bar.table.hm.sorted)
rownames(bar.table.hm.sorted) <- rns

to.plot <- melt(bar.table.hm.sorted, varnames = c("barcode", "MULTI_class"))
to.plot$final_call <- unname(final.calls.rescued[as.character(to.plot$barcode)])
#to.plot$cluster <- factor(g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster, levels=cluster_order)
to.plot$cluster <- g_meta[match(paste0(to.plot$barcode, '-1'), g_meta$barcode)]$cluster
to.plot <- to.plot[order(match(to.plot$cluster, cluster_order)),]
to.plot$barcode <- factor(to.plot$barcode, levels=unique(as.character(to.plot$barcode)))
to.plot$cluster <- factor(to.plot$cluster, levels=cluster_order)

options(repr.plot.width=20, repr.plot.height=10)

p1 <- ggplot(to.plot, aes(MULTI_class, barcode, fill= value)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  scale_fill_continuous(type = "viridis") +
  theme_classic() +
  theme(legend.position = "None",
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
p2 <- ggplot(to.plot, aes(1, barcode, fill= cluster)) + 
  geom_tile() +
  #scale_fill_gradient( trans = 'log2' ) +
  #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
  #scale_fill_continuous(type = "viridis", trans = 'log2') +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
grid.arrange(p1,p2,ncol=2)

for (fc in unique(to.plot$final_call)) {
    
    p1 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    p2 <- ggplot(as.data.table(to.plot)[final_call==fc], aes(1, barcode, fill= cluster)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      #scale_fill_continuous(type = "viridis", trans = 'log2') +
      scale_fill_manual(values = cluster_colours, name = "Cluster") +
      theme_classic() +
      theme(axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    print(grid.arrange(p1,p2,ncol=2,
                       top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    
    p1 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(MULTI_class, barcode, fill= value)) + 
      geom_tile() +
      #scale_fill_gradient( trans = 'log2' ) +
      #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
      scale_fill_continuous(type = "viridis", trans = 'log2') +
      theme_classic() +
      theme(legend.position = "None",
            axis.text.y=element_blank(),
            axis.ticks.y=element_blank(),
            axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
    
    if (nrow(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))]) > 0) {
        p2 <- ggplot(as.data.table(to.plot)[(final_call==fc) & !(is.na(cluster))], aes(1, barcode, fill= cluster)) + 
          geom_tile() +
          #scale_fill_gradient( trans = 'log2' ) +
          #scale_fill_gradient(low="white", high="#2171b5", trans = 'log2')
          #scale_fill_continuous(type = "viridis", trans = 'log2') +
          scale_fill_manual(values = cluster_colours, name = "Cluster") +
          theme_classic() +
          theme(axis.text.y=element_blank(),
                axis.ticks.y=element_blank(),
                axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
        print(grid.arrange(p1,p2,ncol=2,
                           top = textGrob(fc,gp=gpar(fontsize=20,font=3))))
    }
    
    
}

options(repr.plot.width=20, repr.plot.height=15)
to.plot$MULTI_class = factor(to.plot$MULTI_class, levels=gastr_order_short)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_violin() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)
p <- ggplot(as.data.table(to.plot)[MULTI_class == final_call], aes(x=cluster, y=value, fill=cluster)) +
  geom_boxplot() +
  #scale_fill_manual(values = gastr_type_colours, name = "Gastruloid Type") +
  scale_fill_manual(values = cluster_colours, name = "Cluster") +
  theme_classic() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1), legend.position="none") + 
  facet_wrap(~MULTI_class)
print(p)